# DerivKit: DerivativeKit — Advanced Analytic Function Demo

### Summary
This notebook compares the **Adaptive (polynomial-fit)** and **Finite-difference** backends on a more complex analytic function

$f(x) = e^{-x^2}\sin(3x) + 0.1\,x^3$ evaluated at $x_0 = 0.3$.

#### Analytic derivatives (for comparison)

$f'(x) = e^{-x^2}(3\cos(3x) - 2x\sin(3x)) + 0.3\,x^2$

$f''(x) = e^{-x^2}(-9\sin(3x) - 12x\cos(3x) + 4x^2\sin(3x)) + 0.6\,x$

### Usage
If you prefer to run this as a standalone script, it lives in `demo-scripts/02-derivative-kit-advanced.py`.

```bash
$ python demo-scripts/02-derivative-kit-advanced.py
```

### What it does
- Defines the function $f(x)$ and its analytic derivatives.
- Constructs a ``:class:DerivativeKit`` with $x_0 = 0.3$.
- Evaluates numerical derivatives using:
  - Order 1: Adaptive with a user-supplied uniform grid `(grid=("offsets", np.arange(-0.2, 0.25, 0.1)))`
  - Order 2: Adaptive with the default Chebyshev grid and explicit `spacing=0.05`
  - Order 3: Adaptive with `spacing=1/8` and `n_workers=2`
-Prints the numerical results alongside analytic truth values.
-Compares each with a 5-point central finite-difference baseline (or equivalent central stencil) for orders 1–3.


### Notes
- For oscillatory functions, keep the adaptive window local (moderate spacing) and use a modest number of points.
- Adding a small ridge can stabilize higher-order fits.
- When a grid is provided (as in Order 1), `n_points` and `spacing` are ignored for that call.
- The adaptive backend automatically rescales the polynomial basis for numerical stability.

### Requirements
`derivkit` installed and importable in your Python environment.


In [ ]:
import numpy as np

from derivkit.derivative_kit import DerivativeKit


def damped_sine_plus_cubic(x: float) -> float:
    """Returns a damped sine plus cubic function for the demo."""
    return np.exp(-x * x) * np.sin(3.0 * x) + 0.1 * x**3


def truth_1_2_3(x: float) -> tuple[float, float, float]:
    """Returns the analytic 1st, 2nd, and 3rd derivatives at x."""
    ex = np.exp(-x * x)
    s3 = np.sin(3.0 * x)
    c3 = np.cos(3.0 * x)
    d1 = ex * (3.0 * c3 - 2.0 * x * s3) + 0.3 * x * x
    d2 = ex * ((4.0 * x * x - 11.0) * s3 - 12.0 * x * c3) + 0.6 * x
    d3 = ex * ((-8.0 * x**3 + 66.0 * x) * s3 + (36.0 * x * x - 45.0) * c3) + 0.6
    return d1, d2, d3


x0 = 0.3  # Point at which to evaluate derivatives
dk = DerivativeKit(function=damped_sine_plus_cubic, x0=x0)  # Create DerivativeKit instance
t1, t2, t3 = truth_1_2_3(x0)  # Analytic derivatives


## Derivative order 1

Here we will use a **user-supplied uniform grid** for the adaptive method. For the finite-difference baseline, we use a supported 5-point central stencil with a user-defined step size.

In [ ]:
h = 0.16   # half-width of the offset grid
step = 0.02  # step size for the uniform grid
t1_offsets = np.arange(-h, h, step)  # uniform offsets around x0

# Adaptive method
d1_ad = dk.differentiate(order=1, grid=("offsets", t1_offsets))  # Adaptive 1st derivative

# Finite-difference method
d1_fd = dk.differentiate(method="finite", order=1, stepsize=5e-4, num_points=5)

print("Order 1 →  analytic, adaptive, finite:")
print(f"  {t1:>12.6f}   {d1_ad:>12.6f}   {d1_fd:>12.6f}")


## Derivative order 2

Here we use the default Chebyshev grid with an explicit spacing for the adaptive method. For the finite-difference baseline, we again use a supported 5-point central stencil with a user-defined step size.

In [ ]:
# Adaptive method
d2_ad = dk.differentiate(order=2, n_points=19, spacing=0.05, base_abs=1e-3)

# Finite-difference method
d2_fd = dk.differentiate(method="finite", order=2, stepsize=5e-4, num_points=5)

print("\nOrder 2 →  analytic, adaptive, finite:")
print(f"  {t2:>12.6f}   {d2_ad:>12.6f}   {d2_fd:>12.6f}")


## Derivative order 3

Here we use the default Chebyshev grid where spacing=1/8 sets the half-width of the sampling window around x0, with symmetric offsets. We also specify `n_workers=2` to demonstrate parallel evaluation. For the finite-difference baseline, we again use a supported 5-point central stencil with a user-defined step size.

In [ ]:
# Adaptive method
d3_ad = dk.differentiate(order=3, n_points=19, spacing=1/8, n_workers=2)

# Finite-difference method
d3_fd = dk.differentiate(method="finite", order=3, stepsize=5e-4, num_points=5)

print("\nOrder 3 →  analytic, adaptive, finite:")
print(f"  {t3:>12.6f}   {d3_ad:>12.6f}   {d3_fd:>12.6f}")


## Print summary table

In [ ]:
thick = "=" * 62
thin = "-" * 62
print("\n" + thick)
title = "f(x) = exp(-x^2) * sin(3x) + 0.1 * x^3"
print(title)
print(f"x0 = {x0:.4f}")
print(thin)
print("Order   Analytic          Adaptive          Finite")
print(thin)
print(f"  1   {t1:>12.6f}   {d1_ad:>12.6f}   {d1_fd:>12.6f}")
print(f"  2   {t2:>12.6f}   {d2_ad:>12.6f}   {d2_fd:>12.6f}")
print(f"  3   {t3:>12.6f}   {d3_ad:>12.6f}   {d3_fd:>12.6f}")
print(thick)
